# Importing Gaia data to Firefly

*Building off of the tutorial here: http://gea.esac.esa.int/archive-help/index.html *

*The main table is gaiadr2.gaia_source, and [here](http://gea.esac.esa.int/archive/documentation/GDR2/Gaia_archive/chap_datamodel/sec_dm_main_tables/ssec_dm_gaia_source.html) is the description of the columns*

In [1]:
%load_ext autoreload
%autoreload 2
from FIREreader import FIREreader

/Users/ageller/anaconda/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import astropy.units as u
from astropy.coordinates import ICRS
from astropy.coordinates.sky_coordinate import SkyCoord
from astroquery.gaia import Gaia
import numpy as np

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


*Do the following to load and look at the available Gaia table names:*

In [3]:
tables = Gaia.load_tables(only_names=True)
for table in (tables):
    print (table.get_qualified_name())

Retrieving tables...
Parsing tables...
Done.
public.public.dual
public.public.hipparcos
public.public.hipparcos_newreduction
public.public.hubble_sc
public.public.igsl_source
public.public.igsl_source_catalog_ids
public.public.tycho2
tap_schema.tap_schema.columns
tap_schema.tap_schema.key_columns
tap_schema.tap_schema.keys
tap_schema.tap_schema.schemas
tap_schema.tap_schema.tables
gaiadr1.gaiadr1.aux_qso_icrf2_match
gaiadr1.gaiadr1.ext_phot_zero_point
gaiadr1.gaiadr1.allwise_best_neighbour
gaiadr1.gaiadr1.allwise_neighbourhood
gaiadr1.gaiadr1.gsc23_best_neighbour
gaiadr1.gaiadr1.gsc23_neighbourhood
gaiadr1.gaiadr1.ppmxl_best_neighbour
gaiadr1.gaiadr1.ppmxl_neighbourhood
gaiadr1.gaiadr1.sdss_dr9_best_neighbour
gaiadr1.gaiadr1.sdss_dr9_neighbourhood
gaiadr1.gaiadr1.tmass_best_neighbour
gaiadr1.gaiadr1.tmass_neighbourhood
gaiadr1.gaiadr1.ucac4_best_neighbour
gaiadr1.gaiadr1.ucac4_neighbourhood
gaiadr1.gaiadr1.urat1_best_neighbour
gaiadr1.gaiadr1.urat1_neighbourhood
gaiadr1.gaiadr1.cepheid

In [4]:
gaiadr2_table = Gaia.load_table('gaiadr2.gaia_source')
for column in (gaiadr2_table.get_columns()):
    print(column.get_name())

Retrieving table 'gaiadr2.gaia_source'
Parsing table 'gaiadr2.gaia_source'...
Done.
solution_id
designation
source_id
random_index
ref_epoch
ra
ra_error
dec
dec_error
parallax
parallax_error
parallax_over_error
pmra
pmra_error
pmdec
pmdec_error
ra_dec_corr
ra_parallax_corr
ra_pmra_corr
ra_pmdec_corr
dec_parallax_corr
dec_pmra_corr
dec_pmdec_corr
parallax_pmra_corr
parallax_pmdec_corr
pmra_pmdec_corr
astrometric_n_obs_al
astrometric_n_obs_ac
astrometric_n_good_obs_al
astrometric_n_bad_obs_al
astrometric_gof_al
astrometric_chi2_al
astrometric_excess_noise
astrometric_excess_noise_sig
astrometric_params_solved
astrometric_primary_flag
astrometric_weight_al
astrometric_pseudo_colour
astrometric_pseudo_colour_error
mean_varpi_factor_al
astrometric_matched_observations
visibility_periods_used
astrometric_sigma5d_max
frame_rotator_object_type
matched_observations
duplicated_source
phot_g_n_obs
phot_g_mean_flux
phot_g_mean_flux_error
phot_g_mean_flux_over_error
phot_g_mean_mag
phot_bp_n_obs
ph

### Next, we retrieve the data 

*Note: The query to the archive is with ADQL (Astronomical Data Query Language). *

In [5]:
#NOTE: not all sources have RVs
#cmd = "SELECT TOP 100 ra, dec, parallax, pmra, pmdec, radial_velocity FROM gaiadr2.gaia_source"

#NOTE: It looks like there are 
#480235 sources with g <=10 and with parallaxes
#1236351 sources with g <=11 and with parallaxes
#I could only get 3000000 for g<=12.  I wonder if that is the limit?
#I think this is already ordered by gmag, so by default this will pull the brightest first
N = 2e5
gmax = 12

cmd = "SELECT TOP " + str(np.int(N))+" \
ra, dec, parallax, pmra, pmdec, radial_velocity, teff_val, phot_g_mean_mag \
FROM gaiadr2.gaia_source \
WHERE phot_g_mean_mag<=" + str(gmax) +" \
AND parallax IS NOT NULL "#\
#AND radial_velocity IS NOT NULL"
print(cmd)

#synchronous commands are OK for jobs with < 2000 output rows
#job = Gaia.launch_job(cmd, dump_to_file=False) 

#asynchronous commands for larger files
job = Gaia.launch_job_async(cmd, dump_to_file=False)

print (job)

SELECT TOP 200000 ra, dec, parallax, pmra, pmdec, radial_velocity, teff_val, phot_g_mean_mag FROM gaiadr2.gaia_source WHERE phot_g_mean_mag<=12 AND parallax IS NOT NULL 


Query finished.
<Table masked=True length=200000>
      name       dtype     unit    format                description                 n_bad
--------------- ------- ---------- ------ ------------------------------------------ -----
             ra float64        deg {!r:>}                            Right ascension     0
            dec float64        deg {!r:>}                                Declination     0
       parallax float64        mas {!r:>}                                   Parallax     0
           pmra float64 mas.yr**-1 {!r:>} Proper motion in right ascension direction     0
          pmdec float64 mas.yr**-1 {!r:>}     Proper motion in declination direction     0
radial_velocity float64   km.s**-1 {!r:>}                            Radial velocity 62006
       teff_val float32          K {!r:>}              stellar effective temperature   301
phot_g_mean_mag float32        mag {!r:>}                      G-band mean magnitude     0
Jobid: 1525975251384O
Phase: COMPLETED
O

*Inspect the output table and number of rows:*

In [6]:
r = job.get_results()
print(len(r))
print(r)

200000
        ra                 dec         ...  teff_val phot_g_mean_mag
       deg                 deg         ...     K           mag      
------------------ ------------------- ... --------- ---------------
 274.4061143187362  -36.76240607839242 ... 4732.6665       1.9249064
109.28559761663782 -37.097446909729776 ... 4732.6665        1.970477
311.55470104855004   33.97165785643453 ... 4732.6665       2.0226774
 296.5649839440163  10.613253300439279 ... 4732.6665       2.0403643
236.06755715935202   6.425826685796823 ... 4732.6665       2.1248496
226.01721301839848 -25.282153195983703 ...    4710.0       2.1342418
 275.2486775206737  -29.82821901708563 ... 4732.6665       2.1481218
 59.50763338363228 -13.508997297697881 ... 4732.6665       2.1506112
 261.3248898046725  -55.52999303063296 ... 4732.6665       2.1833172
155.58177692208324    41.4996586971537 ... 4732.6665       2.1958747
               ...                 ... ...       ...             ...
298.72293313370795  -2.0597

### Convert these ra, dec, parallax coordinates to 3D cartesian

In [7]:
#it is odd that some of the parallaxes are negative!
xx = np.where(r['parallax'] < 0)[0]
print("Number, fraction where parallax < 0 :", len(xx), len(xx)/len(r['parallax']))
dist = np.abs((r['parallax']).to(u.parsec, equivalencies=u.parallax()))

#this only contains the position, and in cartesian it is much faster
#astroC = SkyCoord(ra=r['ra'], dec=r['dec'], distance=dist).cartesian #transform_to(coord.Galactocentric)

#this would account for velocities also
pmra = np.array([x for x in r['pmra']]) * u.mas/u.yr
pmdec = np.array([x for x in r['pmdec']]) * u.mas/u.yr
rv = np.array([x for x in r['radial_velocity']]) * u.km/u.s 
xx = np.where(np.isnan(r['radial_velocity']) )[0]
print("Number, fraction without RVs :", len(xx), len(xx)/len(r['radial_velocity']))
#################
#################
# what should we about missing RV data?
#################
#################
for i in xx:
    rv[i] = 0*u.km/u.s

#NOTE: without RVs, the velocities are not correct
astroC = ICRS(ra=r['ra'], dec=r['dec'], distance=dist, pm_ra_cosdec=pmra, pm_dec=pmdec, radial_velocity=rv)

print(astroC.cartesian)
print(astroC.velocity)

Number, fraction where parallax < 0 : 428 0.00214


/Users/ageller/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Warning: converting a masked element to nan.
  if sys.path[0] == '':


Number, fraction without RVs : 62006 0.31003
[(  3.1315411 , -40.64130966,  -30.45201359),
 (-69.37069534, 198.25135221, -158.83565473),
 ( 12.74090792, -14.37330244,   12.94170654), ...,
 ( 48.82387921, -65.27626066,  -86.99402657),
 (162.33831139, -90.20258624,  374.40406739),
 ( 95.97674132, -47.23484563,  -38.25681209)] pc
[(-33.55327794,  21.29293495, -31.86804804),
 ( 16.67902548,   7.65774824,   2.27356602),
 ( 17.50045152,  40.82817405,  28.11561606), ...,
 ( -9.51045427, -41.47291497, -42.84546696),
 (  5.09987414,   4.39743501,  -1.15181693),
 ( 13.84408705, -20.2812323 , -28.05847496)] km / s


*Now put these in the format expected by FIREreader, and convert to cartesian*

In [8]:
p = astroC.cartesian
v = astroC.velocity
coordinates = np.dstack((p.x.to(u.pc).value, p.y.to(u.pc).value, p.z.to(u.pc).value)).squeeze()
velocities = np.dstack((v.d_x.to(u.km/u.s).value, v.d_y.to(u.km/u.s).value, v.d_z.to(u.km/u.s).value)).squeeze()
Teff = np.array(r['teff_val'])
gMag = np.array(r['phot_g_mean_mag'])
 
print(coordinates[0:10])
print(velocities[0:10])
print(min(Teff), max(Teff))
print(min(gMag), max(gMag))

[[    3.1315411    -40.64130966   -30.45201359]
 [  -69.37069534   198.25135221  -158.83565473]
 [   12.74090792   -14.37330244    12.94170654]
 [  160.33843567  -320.67781207    67.18262268]
 [  -14.08974221   -20.94212353     2.84272035]
 [  -79.24365749   -82.10856      -53.89665179]
 [  108.7320149  -1183.62180629  -681.49846896]
 [   26.02758967    44.19954352   -12.32302709]
 [  -31.68304766  -207.65290788  -305.97646536]
 [  -69.26527358    31.44668857    67.29982114]]
[[ -33.55327794   21.29293495  -31.86804804]
 [  16.67902548    7.65774824    2.27356602]
 [  17.50045152   40.82817405   28.11561606]
 [  21.02644327   10.37017869   -0.68262904]
 [  14.19712109   -8.7897444     5.61358882]
 [ -25.83807412   38.62141367  -20.84814415]
 [ 230.46615975   88.51491928 -116.96158151]
 [  13.57355724   52.01586083  -41.42106981]
 [  -7.53879845   32.9227632   -21.56262377]
 [  30.10615173   35.05319079   14.60633989]]
3279.0 9803.0
1.9249064 9.118369


### Create the Firefly data files using FIREreader

In [9]:
pname = 'Gaia'

reader = FIREreader()

#set all of these manually
reader.returnParts = [pname] 
reader.names = {pname:pname}
reader.dataDir = pname
reader.JSONfname = pname+'Data'

#create a space for the data in partsDict
reader.partsDict[pname] = dict()

#define the defaults; this must be run first if you want to change the defaults below
reader.defineDefaults()
reader.defineFilterKeys()

#update a few of the options
reader.options['sizeMult'][pname] = 0.001
reader.options['color'][pname] = [1., 1., 1., 1.] 
reader.options['center'] = [0,0,0]

reader.addtodict(reader.partsDict,None,pname,'Coordinates',0,0,vals=coordinates)
reader.addtodict(reader.partsDict,None,pname,'Velocities',0,0,vals=velocities)
reader.addtodict(reader.partsDict,None,pname,'Teff',0,0,vals=Teff, filterFlag = True)
reader.addtodict(reader.partsDict,None,pname,'gMag',0,0,vals=gMag, filterFlag = True)


reader.shuffle_dict()
reader.swap_dict_names()
reader.createJSON()

shuffling ... 
writing JSON files ...
Gaia
Gaia/GaiaDataOptions.json
